## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for ukrainian and russian.

In [4]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [5]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 0.9999999403953552),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095540046691895),
 ('ноябрь', 0.8930034637451172),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557134628296),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401415944099426)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.000000238418579),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810409307479858),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804074287415),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408111333847046),
 ('лютий', 0.8256431818008423)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256801068782806),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632379829883575),
 ('одностороннего', 0.22608883678913116),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003719210624695),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [9]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [10]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [11]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [15]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [16]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411202430725098),
 ('март', 0.8396994471549988),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.832929790019989),
 ('октябрь', 0.8311846852302551),
 ('ноябрь', 0.8278924226760864),
 ('июль', 0.8234528303146362),
 ('август', 0.8120500445365906),
 ('декабрь', 0.8039005398750305)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [36]:
from collections import Counter

a = [0, 1, 2]
counter = Counter(elem for elem in a)
a[:1]
print(counter)
print(counter[40])

Counter({0: 1, 1: 1, 2: 1})
0


In [37]:
uk_ru_test[:10]

[('або', 'либо'),
 ('активний', 'активный'),
 ('актор', 'актер'),
 ('але', 'ж'),
 ('асамблея', 'собрание'),
 ('бабуся', 'бабушка'),
 ('багажник', 'ствол'),
 ('бажати', 'желать'),
 ('башта', 'башня'),
 ('бізнес', 'бизнес')]

In [38]:
uk_emb['asdsads']

KeyError: "Key 'asdsads' not present"

In [48]:
from collections import Counter

def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    mapped_vectors = list(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # _ru_emb = mapping.predict(_uk_emd.reshape(1, -1))
        _ru_emb=mapped_vectors[i]
        print(_ru_emb.reshape(1, -1))
        counter = Counter(elem[0] for elem in ru_emb.most_similar(_ru_emb.reshape(1, -1))[:topn])
        num_matches += float(counter[ru])
    precision_val = num_matches / len(pairs)
    print(precision_val)
    return precision_val


In [49]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

[[ 2.30988860e-03 -4.31191400e-02 -4.31668311e-02 -3.46035138e-02
  -1.71307474e-03 -3.56489047e-02 -4.91968356e-03  1.53636541e-02
   2.78865490e-02  3.03796791e-02 -3.84302996e-03  4.43296582e-02
  -1.11899069e-02  1.83284841e-02  1.90534592e-02 -2.39051972e-02
   5.20489700e-02  1.65667757e-03 -7.67123029e-02  3.39718424e-02
   4.69692536e-02 -5.40799685e-02 -3.05023007e-02 -3.28376517e-03
   4.58410084e-02  5.08171171e-02  3.82831972e-03 -3.13806720e-02
   3.07917520e-02 -4.78141196e-03 -2.07081176e-02  1.61701515e-02
   2.16257144e-02 -8.49262923e-02  1.41674820e-02 -1.57318581e-02
  -5.04166484e-02 -1.71213299e-01 -3.32582146e-02  1.82292499e-02
   3.49863805e-03 -4.00891453e-02 -3.91729735e-03 -1.25835128e-02
  -1.46667007e-02  2.60908622e-03  3.46444547e-02  8.69558528e-02
   3.34567018e-03  5.36744520e-02  3.27023044e-02  1.89812891e-02
   2.82949172e-02  2.61964463e-02 -1.55752208e-02  1.34626478e-02
  -3.01904194e-02 -6.53153658e-02 -5.74788935e-02 -2.76857987e-02
  -8.03057

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

[[ 8.180e-02 -1.100e-03  3.030e-02 -1.830e-02  5.800e-03 -2.110e-02
   9.540e-02 -8.990e-02 -4.140e-02  1.890e-02 -9.000e-03 -6.780e-02
  -1.000e-02 -6.130e-02  3.600e-03  8.190e-02 -4.140e-02 -2.060e-02
  -1.990e-02  1.100e-01 -1.450e-02 -6.000e-02  6.000e-03  1.870e-02
   1.108e-01  4.790e-02 -7.870e-02  1.037e-01  8.650e-02  1.860e-02
  -3.980e-02  1.260e-02  5.290e-02 -2.840e-02  6.990e-02  1.750e-02
  -5.700e-03 -1.930e-02 -2.750e-02  1.200e-02 -9.900e-03 -1.500e-03
   6.360e-02  8.820e-02  8.500e-02 -8.700e-03 -1.136e-01  3.830e-02
   8.900e-03 -2.210e-02 -2.100e-03 -6.740e-02  1.076e-01  6.000e-04
   3.910e-02 -3.610e-02  9.180e-02 -5.270e-02  5.000e-03 -5.000e-02
   1.036e-01  9.210e-02 -2.060e-02  1.330e-02 -3.760e-02 -2.700e-03
   1.570e-02  3.900e-03  8.500e-03 -4.240e-02  1.206e-01 -3.830e-02
  -8.050e-02 -3.200e-03 -2.460e-02 -2.540e-02 -1.970e-02 -1.587e-01
  -4.730e-02 -1.112e-01 -4.630e-02 -8.240e-02  2.870e-02 -6.170e-02
  -1.900e-02  6.880e-02 -5.770e-02 -5.810e-02 -5

In [39]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

0.6356589147286822
0.813953488372093


## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [140]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE

In [141]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [ ]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [186]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [187]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE HERE

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -